In [82]:
from web3 import Web3
from bit import wif_to_key
from constants import *
import subprocess 
import json
import pandas as pd
from web3.middleware import geth_poa_middleware
from eth_account import Account
w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/1fa351728e874efcac3aabd55f70a089"))

In [83]:
php = 'php ./hd-wallet-derive/hd-wallet-derive.php'
coin = 'BTC-test'
format1 = 'json'
cmd = f'{php} -g --mnemonic="{mnem}" --cols=path,address,privkey,pubkey --coin={coin} --format={format1}'

In [84]:
def getKeys(cmd): 
    p = subprocess.Popen(cmd, 
        stdout=subprocess.PIPE, shell=True) 
    (output, err) = p.communicate()
    p_status = p.wait()
    keys = json.loads(output)
    return keys

In [85]:
results = []
coins = ['BTC-test', 'ETH']
for coin in coins:
    keys = getKeys(cmd)
    for key in keys:
        results.append({"Coin": coin, "Address": key["address"], "PrivKey": key["privkey"], "PubKey": key["pubkey"]})

In [86]:
print(results)

[{'Coin': 'BTC-test', 'Address': 'mzywF5C5pcD3dJBXHSsePjZqdNLYecaV2n', 'PrivKey': 'cSMqs1XyhhaEonQpuj2xr7ocejMPAFcVnsZGfcEmq2q6bUuuV4Xe', 'PubKey': '037159c6d63105a9e371e2d25465e0dea39d415b0ad73bc0a122afb987778f11f8'}, {'Coin': 'BTC-test', 'Address': 'mp5o9RvaiV3W3yMDg1HBSzhRuwG9HKoG2U', 'PrivKey': 'cQmbzzU4herEG4sMD7AdQQVqzt5HtrDLWYch1PMonuUTvtFsJJtA', 'PubKey': '0352e523fbb0f86218d3c6b3ce5d4eb7d96069186367a2b8aee1499b7e84780ee3'}, {'Coin': 'BTC-test', 'Address': 'mgDKV2hCp1eQmbtTZnFSR6Zanudv7ts2JC', 'PrivKey': 'cTMmhYPZF9iEhXMRNUWhpHKjyUkTrAFR1doLZ3z95dfoUR1nrLHc', 'PubKey': '03f464486ea02f27e384e1f696ad677e5f22603ce9af796c37c61d40093ea5883e'}, {'Coin': 'BTC-test', 'Address': 'mnu7ERZFm3qycfPqNPKXBgUPyE2XGv4A7M', 'PrivKey': 'cRRTeWT6dL96X6zegGWbkon2UjoYz3XhKYh95vFWgH9sPM9uFhvS', 'PubKey': '03feb685ae1dbd6932f9f868feb7f573c8bdf1c3192f9adebefd4df133f1a51e52'}, {'Coin': 'BTC-test', 'Address': 'muNDTCyaPryoXYzFVn5xQubBLzip7dKAsF', 'PrivKey': 'cMgW9BYwBtwBkwuZ8zUj6hvjSUoWMzNNTpJapkYQ7EaTy

In [87]:
df = pd.DataFrame(results)

In [ ]:
def getbal(row):
    privKey = row["PrivKey"]
    address = row["Address"]
    coin = row["Coin"]
    print(coin, address)
    if coin == 'BTC-test':
        key = wif_to_key(privKey)
        bal = key.get_balance('btc')
    else:
        bal = w3.eth.getBalance(address)
    return bal
    
df["bal"] = df.apply(getbal, axis=1)

In [ ]:
df2 = df[['Coin', 'Address', 'bal']]
df2